In [ ]:
#NOTE code is from 2024: Does not work on current website.

import requests
from bs4 import BeautifulSoup
import csv

print("NOTE that this code is from 2024: Does not work on the current website.")

# Base URL for undergraduate search
base_url = "https://www.bristol.ac.uk/study/undergraduate/search/"

# Request the base URL page
response = requests.get(base_url)
response.raise_for_status()  # ths flags bad responses if page doesn't load

# Parse the HTML content of the main page
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links starting with /study/undergraduate/2025/
links = soup.find_all('a', href=True)
filtered_links = [link['href'] for link in links if link['href'].startswith('/study/undergraduate/2025/')]
full_links = ["https://www.bristol.ac.uk" + link for link in filtered_links]

# Function to find fee and duration on the course page
def find_fee_and_duration_on_page(url):
    try:
        page_response = requests.get(url)
        page_response.raise_for_status()
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        # Find the title of the page and exclude text after the first pipe
        full_title = page_soup.title.string.strip()
        title = full_title.split('|')[0].strip()
        
        # Initialise fee and duration fields
        fee = "Fee not found"
        duration = "Duration not found"
        
        # Fee is in the <dd> HTML tags
        fee_elements = page_soup.find_all('dd')
        for element in fee_elements:
            if "per year, international students" in element.text:
                fee = element.text.strip()
                break

        # Find  prog duration in HTML
        duration_label = page_soup.find('dt', text="Programme duration")
        if duration_label:
            duration_element = duration_label.find_next('dd')
            if duration_element:
                duration = duration_element.text.strip()

        return title, fee, duration
    # Handle errors
    except Exception as e:
        return "Error", f"Failed to process {url}: {e}", "Error"

# set up CSV file to save output
with open('course_fees_and_durations.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Title', 'Fee', 'Duration']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    # Loop through all filtered links, find the fee and duration
    for link in full_links:
        title, fee, duration = find_fee_and_duration_on_page(link)
        writer.writerow({'Title': title, 'Fee': fee, 'Duration': duration})
        print(f"Title: {title}")
        print(f"Fee: {fee}")
        print(f"Duration: {duration}\n")
